In [1]:
import sys, os
import pickle
sys.path.insert(0, '/home/cvds_lab/yury/mxt-experiments/mlperf/recommendation/pytorch')
from alias_generator import AliasSample
from convert import generate_negatives
import torch
from tqdm import tqdm
import torch.nn as nn
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [10]:
data_dir = '/home/cvds_lab/yury/mxt-experiments/mlperf/recommendation/pytorch/ml-20mx16x32'

sampler_cache = os.path.join(data_dir, "alias_tbl_16x32_cached_sampler.pkl")
with open(sampler_cache, "rb") as f:
    sampler, pos_users_full, pos_items_full, nb_items_full, pck = pickle.load(f)
print("Done")
print(nb_items_full)
print(len(pos_users_full))
print(len(pos_items_full))

Done
855776
1223962043
1223962043


In [11]:
max_user = pos_users_full.max()
print("max user: {}".format(max_user))
max_item = pos_items_full.max()
print("max item: {}".format(max_item))

max user: 2197224
max item: 855775


In [19]:
N = 10000
p = np.random.choice(np.arange(len(pos_users_full)), size=N, replace=False)

In [23]:
nb_items = nb_items_full
pos_users = pos_users_full[p]
pos_items = pos_items_full[p]
print("max user: {}".format(pos_users.max()))
print("max item: {}".format(pos_items.max()))

max user: 2196900
max item: 855453


In [24]:
nb_users = len(sampler.num_regions)
train_users = torch.from_numpy(pos_users).type(torch.LongTensor)
train_items = torch.from_numpy(pos_items).type(torch.LongTensor)

In [25]:
# produce things not change between epoch
# mask for filtering duplicates with real sample
# note: test data is removed before create mask, same as reference
# create label
negative_samples = 4
train_label = torch.ones_like(train_users, dtype=torch.float32)
neg_label = torch.zeros_like(train_label, dtype=torch.float32)
neg_label = neg_label.repeat(negative_samples)
train_label = torch.cat((train_label,neg_label))
del neg_label

In [6]:
from neumf import NeuMF

# Create model
model = NeuMF(2197225, 855776,
              mf_dim=64, mf_reg=0.,
              mlp_layer_sizes=[256, 256, 128, 64],
              mlp_layer_regs=[0. for i in [256, 256, 128, 64]])

# Add optimizer and loss to graph
params = model.parameters()

learning_rate = 0.001
beta1 = 0.9
beta2 = 0.999
eps = 1e-8
optimizer = torch.optim.Adam(params, lr=learning_rate, betas=(beta1, beta2), eps=eps)
criterion = nn.BCEWithLogitsLoss(reduction = 'none') # use torch.mean() with dim later to avoid copy to host

model = model.cuda()
criterion = criterion.cuda()
        
print(model)


:::MLPv0.5.0 ncf 1569736180.287664652 (/home/cvds_lab/yury/mxt-experiments/mlperf/recommendation/pytorch/neumf.py:19) model_hp_mf_dim: 64

:::MLPv0.5.0 ncf 1569736193.064679861 (/home/cvds_lab/yury/mxt-experiments/mlperf/recommendation/pytorch/neumf.py:27) model_hp_mlp_layer_sizes: [256, 256, 128, 64]
NeuMF(
  (mf_user_embed): Embedding(2197225, 64)
  (mf_item_embed): Embedding(855776, 64)
  (mlp_user_embed): Embedding(2197225, 128)
  (mlp_item_embed): Embedding(855776, 128)
  (mlp): ModuleList(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
  )
  (final_mf): Linear(in_features=64, out_features=1, bias=True)
  (final_mlp): Linear(in_features=64, out_features=1, bias=True)
)


In [26]:
negatives = generate_negatives(sampler, negative_samples, train_users.numpy())
negatives = torch.from_numpy(negatives)
neg_users = negatives[:, 0]
neg_items = negatives[:, 1]
epoch_users = torch.cat((train_users,neg_users))
epoch_items = torch.cat((train_items,neg_items))
del neg_users, neg_items

# shuffle prepared data and split into batches
epoch_indices = torch.randperm(len(epoch_users))
epoch_size = len(epoch_indices)
epoch_users = epoch_users[epoch_indices]
epoch_items = epoch_items[epoch_indices]
epoch_label = train_label[epoch_indices]

In [27]:
cal_set_f = os.path.join(data_dir, 'cal_set')
# f = open(cal_set_f, 'w')
cal_set_dict = {'users': epoch_users, 'items': epoch_items, 'labels': epoch_label}
torch.save(cal_set_dict, cal_set_f)

In [28]:
torch.load(cal_set_f)

{'items': tensor([638857, 531419, 625382,  ..., 162897, 678570, 546731]),
 'labels': tensor([0., 0., 0.,  ..., 0., 0., 0.]),
 'users': tensor([1222745,   74998, 1579138,  ...,  363161, 1196984, 1181740])}

In [7]:
batch_size = 256
local_batch = batch_size
epochs = 1
for epoch in range(epochs):
    negatives = generate_negatives(sampler, negative_samples, train_users.numpy())
    negatives = torch.from_numpy(negatives)
    neg_users = negatives[:, 0]
    neg_items = negatives[:, 1]
    epoch_users = torch.cat((train_users,neg_users))
    epoch_items = torch.cat((train_items,neg_items))
    del neg_users, neg_items
    
    # shuffle prepared data and split into batches
    epoch_indices = torch.randperm(len(epoch_users))
    epoch_size = len(epoch_indices)
    epoch_users = epoch_users[epoch_indices]
    epoch_items = epoch_items[epoch_indices]
    epoch_label = train_label[epoch_indices]
    epoch_users_list = epoch_users.split(local_batch)
    epoch_items_list = epoch_items.split(local_batch)
    epoch_label_list = epoch_label.split(local_batch)
    
    # only print progress bar on rank 0
    num_batches = (epoch_size + batch_size - 1) // batch_size
    qbar = tqdm(range(num_batches))
    
#     for i in qbar:
        # selecting input from prepared data
#         user = epoch_users_list[i].cuda()
#         item = epoch_items_list[i].cuda()
#         label = epoch_label_list[i].view(-1,1).cuda()

#         for p in model.parameters():
#             p.grad = None

#         outputs = model(user, item)
#         loss = criterion(outputs, label).float()
#         loss = torch.mean(loss.view(-1), 0)
#         print(loss.item())

#         loss.backward()
#         optimizer.step()

  0%|          | 0/196 [00:00<?, ?it/s]

In [16]:
epoch_users_list[0]

tensor([ 6,  5, 14,  5, 12,  2, 13,  3,  3, 13, 14,  6,  4,  3,  9,  3,  9,  6,
         4,  2,  2, 13,  6,  0, 12, 11,  6,  9,  1,  0,  9,  5,  9, 10,  6, 13,
        13,  7,  4,  4,  9, 12,  3,  2,  5,  3,  3, 11, 11,  7,  6,  6, 12,  9,
         9,  0,  6,  5,  3, 13,  6,  3,  3,  7,  3,  8,  6,  9,  6, 13,  3, 12,
         9,  9,  9,  1, 14,  1, 10,  3, 13,  9,  2, 11,  3,  2,  3,  7,  4,  9,
         0, 10, 13,  3,  1,  0, 14, 13,  6,  6,  6, 11,  2,  3, 12,  5, 12,  8,
         6,  6,  4, 14,  4,  5,  8,  9, 12,  4,  2,  9, 13,  9,  7,  7,  7, 10,
        12,  6,  0,  9,  6, 12, 11, 12,  2,  6,  3,  2, 11,  9,  5, 13,  0,  3,
         4,  3,  1,  6,  6, 11,  3,  2,  3,  6,  4, 13, 10,  8,  5, 13, 12,  8,
         7,  6,  8, 11,  5,  3,  1,  5,  7,  3,  5,  0,  2,  4, 12,  4, 12,  3,
         7,  9,  1, 11, 10,  3,  1, 12, 15,  8, 13,  1,  7, 10,  6, 11,  9,  3,
        12,  9,  3,  2,  1,  1, 13,  7, 13, 11, 12,  1, 10, 11, 13,  1, 14,  8,
        11, 11,  3,  0,  3, 14,  7,  6, 

In [17]:
epoch_items_list[0]

tensor([738434, 435846, 151808, 832544, 420195, 177795, 382987, 563540, 696714,
        150602, 359062, 843844, 534328, 398248, 736186, 241708, 168527, 677968,
        130986, 313454,  60469, 701909, 582572, 215296, 614220, 377902, 203404,
        615900, 296711,  39836, 389947, 594673, 300492, 489974, 412205, 498236,
        728268, 510405, 394506, 188835, 443500, 432449, 234825, 255006, 694706,
        741851, 375773, 320001, 537916,  17696,  69316, 581796, 203522, 670856,
        851786, 629876, 789018, 766496, 498923, 339141, 322131, 611526, 506599,
        187971, 356328, 266931,  52219, 622618, 402324, 547895, 628472, 637481,
         35095, 770339, 121111, 283057, 616152, 660983,   1013, 137968, 457527,
        387486, 232585, 575139, 248926,   1937, 669186, 386263, 194287, 183478,
        419780, 245735, 319916, 376053, 192176, 148524, 384232, 625605,  76647,
        217490, 665915, 394617, 113411, 390937, 417765, 619537, 364686,  96900,
        230661, 411551,  95168, 507191, 

In [27]:
nz = epoch_label_list[0].nonzero().flatten()
print(epoch_users_list[0][nz])
print(epoch_items_list[0][nz])

tensor([13,  3,  3,  9,  0, 11,  9,  9,  4, 11,  7,  9,  3,  2,  7,  0,  3,  1,
        11,  3,  8,  6,  4,  5,  8,  0, 11,  3,  5,  3,  1,  9, 10,  1, 12,  6,
         9,  1,  8, 11,  3, 14,  7,  9, 12,  4, 13, 15])
tensor([382987, 398248, 241708, 168527, 215296, 377902, 615900, 389947, 188835,
        320001, 187971, 387486, 248926,   1937, 386263, 419780, 376053, 192176,
        394617, 390937,  96900, 411551, 823957, 134680, 235354, 100530, 593402,
        241639, 193373, 385732, 161483, 713095, 226755, 187416,  30025, 212724,
        186456, 153183, 181915, 191639, 247104, 210927, 381100, 594502,  43808,
        228163, 379052,  20775])
